# Forward kinematics for the modified Tinkerkit Braccio

## Define the basic DH transformation matrix

In [3]:
from sympy import *
# define the variables we are using within 
var('d,a,alpha,delta,Rz,Rx,Tz,Tx')

Rx=Matrix([[1,0,0,0],
          [0,cos(alpha),-sin(alpha),0],
          [0,sin(alpha),cos(alpha),0],
          [0,0,0,1]])
Rz=Matrix([[cos(delta),-sin(delta),0,0],
          [sin(delta),cos(delta),0,0],
          [0,0,1,0],
          [0,0,0,1]])
Tz=Matrix([[1,0,0,0],
          [0,1,0,0],
          [0,0,1,d],
          [0,0,0,1]])
Tx=Matrix([[1,0,0,a],
          [0,1,0,0],
          [0,0,1,0],
          [0,0,0,1]])
TM=Tz.multiply(Rz).multiply(Tx).multiply(Rx)
TM

Matrix([
[cos(delta), -sin(delta)*cos(alpha),  sin(alpha)*sin(delta), a*cos(delta)],
[sin(delta),  cos(alpha)*cos(delta), -sin(alpha)*cos(delta), a*sin(delta)],
[         0,             sin(alpha),             cos(alpha),            d],
[         0,                      0,                      0,            1]])

## Definition of the transformation function for each link

In [5]:
# function for converting degrees to radiant
def d2r(deg):
    return deg/180*pi


def Tv1n0(delta1):
    var('delta1,d1,alpha1,a1')
    #DH-Parameter
    d1=70
    #convert angle from degree to rad and compensate the offset from the home position
    delta1=d2r(delta1) + pi - 0
    a1=0    
    alpha1=pi/2
    return TM.subs({delta:delta1,d:d1,alpha:alpha1,a:a1})

def Tv2n1(delta2):
    var('delta2,d2,alpha2,a2')
    #DH-Parameter
    d2=0
    #convert angle from degree to rad and compensate the offset from the home position
    delta2=d2r(delta2)
    a2=120
    alpha2=0
    return TM.subs({delta:delta2,d:d2,alpha:alpha2,a:a2})

def Tv3n2(delta3):
    var('delta3,d3,alpha3,a3')
    #DH-Parameter
    d3=0
    #convert angle from degree to rad and compensate the offset from the home position
    delta3=d2r(delta3) - pi/2
    a3=120    
    alpha3=0
    return TM.subs({delta:delta3,d:d3,alpha:alpha3,a:a3})

def Tv4n3(delta4):
    var('delta4,d4,alpha4,a4')
    #DH-Parameter
    d4=0
    #convert angle from degree to rad and compensate the offset from the home position
    delta4=d2r(delta4)-pi/2
    a4=80    
    alpha4=-pi/2
    return TM.subs({delta:delta4,d:d4,alpha:alpha4,a:a4})

def Tv5n4(delta5):
    var('delta5,d5,alpha5,a5')
    #DH-Parameter
    d5=150
    #convert angle from degree to rad and compensate the offset from the home position
    delta5=d2r(delta5)-pi/2
    a5=0    
    alpha5=0
    return TM.subs({delta:delta5,d:d5,alpha:alpha5,a:a5})

#################################################
# add here the other transformation functions....
#################################################


# Test:
# define a point in the grippers CS (CS 5)
P5=Matrix([0,0,0,1])
# ... and calculate this points' coordinates in CS0
P0=Tv1n0(0)*Tv2n1(90)*Tv3n2(180)*Tv4n3(90)*Tv5n4(0)*P5
N(P0)


Matrix([
[200.0],
[    0],
[ 40.0],
[  1.0]])

## Function for calculating the TCP-position from the joint angles using the forward transformation

In [7]:
# works only, if other transformation functions are defined ;-)
def BraccioForward(base, shoulder, elbow,wrist,twist):
    return Tv1n0(base)*Tv2n1(shoulder)*Tv3n2(elbow)*Tv4n3(wrist)*Tv5n4(twist)

# Test
P0=BraccioForward(0,90,180,90,0)*Matrix([0,0,0,1])
N(P0)

Matrix([
[200.0],
[    0],
[ 40.0],
[  1.0]])

## Test the forward transformation with the Braccio

In [9]:
import serial
import time
s = serial.Serial('COM7', 115200, timeout=5) #zu AHuse: COM3
time.sleep(3)

# Move to the Home position

In [11]:
s.write(b'P90,90,0,90,0,100,30\n')
print(s.readline().decode())

OK



Define a function which calculates the gripper's tip position and actuates the robot with the given angles

In [13]:
def TestBraccioForward(base, shoulder, elbow,wrist,twist):
    P0=BraccioForward(base, shoulder, elbow,wrist,twist)*Matrix([0,0,0,1])
    print(pretty(N(P0)))
    command="P"+str(int(base))+"," \
                +str(int(shoulder))+"," \
                +str(int(elbow))+","\
                +str(int(wrist))+","\
                +str(int(twist))+",130,50\n"
    s.write(command.encode('ascii'))
    return #Tv1n0(base)*Tv2n1(shoulder)*Tv3n2(elbow)*Tv4n3(wrist)*Tv5n4()
    


Put in values for the joint angles, the robot should move to the calculated position. Check with the caliper, if the position is correct.

In [25]:
TestBraccioForward(0,90,180,90,0)

⎡200.0⎤
⎢     ⎥
⎢  0  ⎥
⎢     ⎥
⎢40.0 ⎥
⎢     ⎥
⎣ 1.0 ⎦


# s.close()